In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
#import pypots
from pypots.data import load_specific_dataset, mcar, masked_fill
from pypots.imputation import SAITS
from pypots.utils.metrics import cal_mae

c:\Users\RUSHI\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
train_directory = "D:/GitHub/ece209as_project/data/UCI_HAR_Dataset/train/Inertial_Signals/"

train_files = ["body_acc_x_train.txt", "body_acc_y_train.txt", "body_acc_z_train.txt", 
        "body_gyro_x_train.txt", "body_gyro_y_train.txt", "body_gyro_z_train.txt", 
        "body_acc_x_train.txt", "body_acc_y_train.txt", "body_acc_z_train.txt"]

train_df = []
train_y = pd.read_csv("D:/GitHub/ece209as_project/data/UCI_HAR_Dataset/train/y_train.txt", header=None)

test_directory = "D:/GitHub/ece209as_project/data/UCI_HAR_Dataset/test/Inertial_Signals/"

test_files = ["body_acc_x_test.txt", "body_acc_y_test.txt", "body_acc_z_test.txt", 
        "body_gyro_x_test.txt", "body_gyro_y_test.txt", "body_gyro_z_test.txt", 
        "body_acc_x_test.txt", "body_acc_y_test.txt", "body_acc_z_test.txt"]

test_df = []
test_y = pd.read_csv("D:/GitHub/ece209as_project/data/UCI_HAR_Dataset/test/y_test.txt", header=None)


for train_file in train_files:
    df = pd.read_csv(train_directory+train_file, delim_whitespace=True, header=None)
    train_df.append(np.array(df))

for test_file in test_files:
    df = pd.read_csv(test_directory+test_file, delim_whitespace=True, header=None)
    test_df.append(np.array(df))

train_X = np.array(train_df).reshape([len(train_y), 128, 9])
test_X = np.array(test_df).reshape([len(test_y), 128, 9])

In [10]:
X_intact, X, missing_mask, indicating_mask = mcar(train_X, 0.1) # hold out 10% observed values as ground truth
X = masked_fill(X, 1 - missing_mask, np.nan)
# Model training. This is PyPOTS showtime. 💪
saits = SAITS(n_steps=128, n_features=9, n_layers=2, d_model=256, d_inner=128, n_head=4, d_k=64, d_v=64, dropout=0.1, epochs=10)
saits.fit(X)  # train the model. Here I use the whole dataset as the training set, because ground truth is not visible to the model.
imputation = saits.impute(X)  # impute the originally-missing values and artificially-missing values
mae = cal_mae(imputation, X_intact, indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)

Model initialized successfully. Number of the trainable parameters: 1332038
epoch 0: training loss 0.2167
epoch 1: training loss 0.1207
epoch 2: training loss 0.1029
epoch 3: training loss 0.0908
epoch 4: training loss 0.0796
epoch 5: training loss 0.0753
epoch 6: training loss 0.0688
epoch 7: training loss 0.0643
epoch 8: training loss 0.0625
epoch 9: training loss 0.0608
Finished training.
